In [1]:
%matplotlib inline
%pylab inline
from time import sleep
import numpy as np 
import scipy
import os
import sys
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, confusion_matrix

from random import randint
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import LSTM
from keras.optimizers import RMSprop

from keras.layers.core import Dense, Dropout, Activation, Flatten


#Questions:
#question: Is there a better way to do the input than just flattening?

#Completed tasks
#Write a normal neural net to try to predict the next measurement with the window method in multiple dimensions (train_baseline_multi)
#Write a deep neural net to try to predict the next measurement with the window method in multiple dimensions (train_baseline_multi)
#Write a Random Forest Regressor to try to predict the next measurement with the window method in multiple dimensions (train_forest)
#Consolidate the pipeline - shape data / create models / validate models 
#Sliding window with a RNN to predict the next value (train_window_regression_rnn)
#Random Forest Classifier to classify FLAG at the next measurement(30 trees, 30 features)
#Neural Network Classifier (1 layer, 512 nodes) 
#Deep Neural Network Classifier (2 layers, 512 nodes) 
#Recurrent Neural Network for classification using window
#Complete all the quizzes and submit a request for MIMIC
#Sanity check for A = B
    #If it doesn't work, make sure the rolling/unrolling is correct
    #Switch order of patient and timestep
    #If it still doesn't work, try only feeding it one previous instead of 20 previous
#Completed today
#Fix the regression
    #Make it so that the matrix predictor gives 0 error
    #Make the other regressors make sense
    #Make all the regressions work
#Make all the classifications work similarly

#--------------------------------------------------------------------------------------------------------------------------------------------
#Regression
    #Write scaffolding for testing variation of len_sequence DONE
    #Fix the data generation DONE
    #Create the graphs
    #Write up Regression where A \neq B
    #Write up Regression where A = B
#Writeup: Classification 
    #Write scaffolding for testing variation of len_sequence 
    #Create the graphs
    #Write up Classification
#Sequence to sequence rnn classification
    #Get a 1-d version working DONE
    #Investigate what happens more closely by reducing the number of people to 2 DONE
    #Get a multiple dimension version working DONE
    #Go back to the toy example and make it work for larger dimensions
    #Bring it back to our synthetic data
#Figure out what we do if data is missing
#Implement it
#--------------------------------------------------------------------------------------------------------------------------------------------
#TODO 
#Investigate if any of our methods can actually predict in the time period where the flag is being flipped: e.g. the previous window has both 0 and 1
#Write a normal neural net to take prediction from rnn and predict 1/0
#Investigate this idea of deleting data
#Add a stateful try where the rnn is an autoencoder ( http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf) and then another LSTM

Populating the interactive namespace from numpy and matplotlib


Using Theano backend.


In [2]:
# fix random seed for reproducibility
np.random.seed(7)
V=np.random.randn(10)
print(V.dot(V), np.linalg.norm(V)**2)

8.33980975239 8.33980975239


In [3]:
#Setup for the generation of the data 
d=10 ##number of measurements per patient per time unit
k=2 ##complexity of the time series
n=200 ## number of patients
len_sequence = 10
sigmaw=0.2##randomness
T=175 ## amount of time measured for each patient (also batch size)
U=np.random.randn(d,k).dot(np.random.randn(k,d))
betashift=np.random.randn(d)/np.sqrt(d)  ## used to check when we switch to the other time series
#normalized U
A=0.9*U/np.linalg.norm(U,ord=2)
#V is also d by d
V=np.random.randn(d,k).dot(np.random.randn(k,d))
#normalized V
B = .9*V/np.linalg.norm(V,ord=2) 
B = A #Set B = A for sanity checking
flag = 1

In [4]:
def generate_data():
    X = np.zeros((n,T,d)) #this represents n patients, in d-dimensions, for T time-steps
    flags = np.zeros((n,T))
    #Generate the data, fill in X, flags
    for j in range(n): #for each patient
        X[j,0,:]=np.random.randn(d)/np.sqrt(d)
        flag=1
        for i in range(1,T): #for each time      
            if flag and ((X[j,i-1,:].dot(betashift))>.5): ##if flag was true and we go above a certain threshold, then set flag=0
                flag = 0
            ## we can think of flag has flag==1 means not critical, flag==0 means critical
            if flag:
                X[j,i,:]=A.dot(X[j,i-1,:])+sigmaw*np.random.randn(d)
                flags[j,i] = 0
            else:
                X[j,i,:]=B.dot(X[j,i-1,:])+sigmaw*np.random.randn(d)
                flags[j, i] = 1
    return(X, flags)

In [5]:
def generate_crazy(len_sequence, X, flags):
    ## X is a n by d by T tensorx
    ## n represents independent trials (think of them as patients or time periods for patients separated)
    ## T represents the time horizon (how long we observe a patient, for now T is the same for everybody, but we can imagine that changing)
    ## d is the number of measurements that we have per patient per time.
    ## flags are num_patients * num_timesteps * dimensionality
    crazy = np.array([X[:, i:i + len_sequence] for i in range(X.shape[1] - len_sequence)])
    crazy_y = np.array([X[:, i + len_sequence] for i in range(X.shape[1] - len_sequence)])
    crazy_flags = np.array([flags[:, i + len_sequence] for i in range(X.shape[1] - len_sequence)])
    return (crazy, crazy_y, crazy_flags)

In [6]:
len_sequence = 1
(X, flags) = generate_data()
(crazy, crazy_y, crazy_flags) = generate_crazy(len_sequence, X, flags)

In [7]:
print(X.shape)
print(flags.shape)
print(crazy.shape)
print(crazy_y.shape)
print(crazy_flags.shape)

#print(flags[6])
#print(crazy_flags[:,6])
#print(X[0,:,0])
#print(crazy[:,0,:,0])
#print(crazy_y[:,0,0])

(200, 175, 10)
(200, 175)
(174, 200, 1, 10)
(174, 200, 10)
(174, 200)


In [ ]:
cra